📝 **Author:** Amirhossein Heydari - 📧 **Email:** <amirhosseinheydari78@gmail.com> - 📍 **Origin:** [mr-pylin/media-processing-workshop](https://github.com/mr-pylin/media-processing-workshop)

---


**Table of contents**<a id='toc0_'></a>    
- [Dependencies](#toc1_)    
- [Load Images](#toc2_)    
  - [Image Degradation](#toc2_1_)    
- [Image Enhancement](#toc3_)    
  - [Spatial Domain: Spatial Filtering](#toc3_1_)    
    - [Linear Filters](#toc3_1_1_)    
      - [Smoothing (Low-Pass) Filters](#toc3_1_1_1_)    
        - [Averaging Filter](#toc3_1_1_1_1_)    
        - [Gaussian Filter](#toc3_1_1_1_2_)    
      - [Sharpening (High Pass) Filters](#toc3_1_1_2_)    
        - [Laplacian Filter](#toc3_1_1_2_1_)    
        - [Unsharp Masking](#toc3_1_1_2_2_)    
        - [High-Boost Filtering](#toc3_1_1_2_3_)    
      - [Edge Detection Filters](#toc3_1_1_3_)    
        - [First-Order Derivative Filters](#toc3_1_1_3_1_)    
        - [Second-Order Derivative Filters](#toc3_1_1_3_2_)    
    - [Nonlinear Filters](#toc3_1_2_)    
      - [Order-Statistic Filters](#toc3_1_2_1_)    
        - [Median Filter](#toc3_1_2_1_1_)    
        - [Maximum and Minimum Filters](#toc3_1_2_1_2_)    
        - [Percentile Filters](#toc3_1_2_1_3_)    
      - [Bilateral Filter](#toc3_1_2_2_)    
      - [Canny Edge Detector](#toc3_1_2_3_)    
      - [Adaptive Filtering](#toc3_1_2_4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Dependencies](#toc0_)

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

In [ ]:
np.set_printoptions(linewidth=120)

In [ ]:
rng = np.random.default_rng(seed=42)

# <a id='toc2_'></a>[Load Images](#toc0_)

In [ ]:
im_1 = cv2.imread("../assets/images/dip_3rd/CH02_Fig0222(b)(cameraman).tif", cv2.IMREAD_GRAYSCALE)
im_2 = cv2.cvtColor(
    cv2.imread("../assets/images/dip_3rd/CH06_Fig0638(a)(lenna_RGB).tif"),
    cv2.COLOR_BGR2RGB,
)

# plot
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 4), layout="compressed")
axs[0].imshow(im_1, vmin=0, vmax=255, cmap="gray")
axs[0].set_title("CH02_Fig0222(b)(cameraman).tif")
axs[1].imshow(im_2, vmin=0, vmax=255)
axs[1].set_title("CH06_Fig0638(a)(lenna_RGB).tif")
plt.show()

## <a id='toc2_1_'></a>[Image Degradation](#toc0_)

- Check out the [**image-degredation.ipynb**](./utils/image-degredation.ipynb) notebook for more information on the topic of degredation.


In [ ]:
def apply_gaussian_noise(img: np.ndarray, mean: float = 0, std: float = 25) -> np.ndarray:
    gaussian_noise = rng.normal(loc=mean, scale=std, size=img.shape)
    return np.clip(img.astype(np.float64) + gaussian_noise, 0, 255).astype(np.uint8)

In [ ]:
def apply_salt_and_pepper_noise(img: np.ndarray, salt_prob: float = 0.05, pepper_prob: float = 0.05) -> np.ndarray:
    new_img = img.copy()

    mask = rng.random(img.shape[:2])
    salt_mask = mask < salt_prob
    pepper_mask = mask > (1 - pepper_prob)

    new_img[salt_mask] = 255
    new_img[pepper_mask] = 0

    return new_img

In [ ]:
def apply_averaging_filter(img: np.ndarray, kernel_size: int) -> np.ndarray:
    kernel = np.ones((kernel_size, kernel_size)) / kernel_size**2
    return cv2.filter2D(img, -1, kernel, borderType=cv2.BORDER_CONSTANT)

In [ ]:
im_1_gaussian_noise = apply_gaussian_noise(im_1, mean=0, std=20)
im_2_gaussian_noise = apply_gaussian_noise(im_2, mean=0, std=50)
im_1_salt_pepper_noise = apply_salt_and_pepper_noise(im_1, salt_prob=0.05, pepper_prob=0.05)
im_2_salt_pepper_noise = apply_salt_and_pepper_noise(im_2, salt_prob=0.05, pepper_prob=0.05)
im_1_blurred = apply_averaging_filter(im_1, kernel_size=3)
im_2_blurred = apply_averaging_filter(im_2, kernel_size=3)

In [ ]:
# plot
images = [
    [im_1, im_1_blurred, im_1_gaussian_noise, im_1_salt_pepper_noise],
    [im_2, im_2_blurred, im_2_gaussian_noise, im_2_salt_pepper_noise],
]
titles = [
    ["Original", "Blurred", "Gaussian Noise", "Salt & Pepper Noise"],
    ["Original", "Blurred", "Gaussian Noise", "Salt & Pepper Noise"],
]
cmaps = [["gray"] * 4, [None] * 4]
fig, axs = plt.subplots(2, 4, figsize=(16, 8), layout="compressed")
for i in range(2):
    for j in range(4):
        axs[i, j].imshow(images[i][j], cmap=cmaps[i][j], vmin=0, vmax=255)
        axs[i, j].set_title(titles[i][j])
        axs[i, j].axis("off")
plt.show()

# <a id='toc3_'></a>[Image Enhancement](#toc0_)

Image enhancement is the procedure of improving the quality for a specific purpose!

- Spatial Domain
  - Intensity Transformation
  - Histogram Processing
  - **Spatial Filtering**
- Frequency Domain
  - Fourier Transform
  - Cosine Transform
- Hybrid Domain
  - Wavelet Transform


## <a id='toc3_1_'></a>[Spatial Domain: Spatial Filtering](#toc0_)

- It is used to **modify** or **enhance** an image by applying an operation to the pixel values within a **defined neighborhood**.
- It processes the image in its **original** form (**pixel values**) rather than **transforming** it into another domain (e.g., **frequency domain**).

<figure style="text-align:center; margin:0;">
  <img src="../assets/images/original/vector/spatial-filtering/spatial-filtering.svg" alt="spatial-filtering.svg" style="max-width:80%; height:auto;">
  <figcaption>Spatial Filtering Concept</figcaption>
</figure>

📝 **Docs**:

- Image Filtering: [docs.opencv.org/master/d4/d86/group__imgproc__filter.html](https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html)
- Operations on arrays: [docs.opencv.org/master/d2/de8/group__core__array.html](https://docs.opencv.org/master/d2/de8/group__core__array.html)
- Signal processing (`scipy.signal`): [docs.scipy.org/doc/scipy/reference/signal.html](https://docs.scipy.org/doc/scipy/reference/signal.html)
- Multidimensional image processing (`scipy.ndimage`): [docs.scipy.org/doc/scipy/reference/ndimage.html](https://docs.scipy.org/doc/scipy/reference/ndimage.html)


### <a id='toc3_1_1_'></a>[Linear Filters](#toc0_)

- The **output** pixel value is a **weighted sum** of the **input** pixel values in the **neighborhood**.
- Linear filtering is mathematically equivalent to **convolution** of the **image** with the **filter mask**.
- Convolution involves **flipping** the mask (**rotating it by 180°**) before applying it, but for **symmetric masks**, this step is often **omitted**.
- Check out the [**convolution.ipynb**](./utils/convolution.ipynb) notebook for comprehensive information on the topic of convolution.


#### <a id='toc3_1_1_1_'></a>[Smoothing (Low-Pass) Filters](#toc0_)

- Used to **reduce noise** and **blur** the image.


##### <a id='toc3_1_1_1_1_'></a>[Averaging Filter](#toc0_)

- Replaces each pixel with the average of its neighborhood pixels.
- For an averaging filter of size $\mathbf{N \times N}$, the mask is a matrix where all elements are equal to $\mathbf{\frac{1}{N^2}}$​:

$$
H_{\text{avg}} = \frac{1}{N^2} \begin{bmatrix}
1 & 1 & \cdots & 1 \\
1 & 1 & \cdots & 1 \\
\vdots & \vdots & \ddots & \vdots \\
1 & 1 & \cdots & 1
\end{bmatrix}_{N \times N}
$$


In [ ]:
kernel_average_3x3 = np.ones((3, 3), dtype=np.float32) / 9
kernel_average_5x5 = np.ones((5, 5), dtype=np.float32) / 25
kernel_average_9x9 = np.ones((9, 9), dtype=np.float32) / 81

# log
print(f"kernel_average_3x3:\n{kernel_average_3x3}\n")
print(f"kernel_average_5x5:\n{kernel_average_5x5}\n")
print(f"kernel_average_9x9:\n{kernel_average_9x9}")

In [ ]:
# using OpenCV
im_1_gauss_averaging_1 = cv2.filter2D(im_1_gaussian_noise, -1, kernel_average_3x3, borderType=cv2.BORDER_CONSTANT)
im_1_gauss_averaging_2 = cv2.filter2D(im_1_gaussian_noise, -1, kernel_average_5x5, borderType=cv2.BORDER_CONSTANT)
im_1_gauss_averaging_3 = cv2.filter2D(im_1_gaussian_noise, -1, kernel_average_9x9, borderType=cv2.BORDER_CONSTANT)

# plot
images = [im_1, im_1_gaussian_noise, im_1_gauss_averaging_1, im_1_gauss_averaging_2, im_1_gauss_averaging_3]
titles = ["Original", "Gaussian Noise", "3x3", "5x5", "9x9"]
fig, axs = plt.subplots(2, 5, figsize=(20, 8), layout="compressed")
for i in range(5):
    axs[0, i].imshow(images[i], cmap="gray", vmin=0, vmax=255)
    axs[0, i].set_title(titles[i])
    axs[0, i].axis("off")
    axs[1, i].imshow(images[i][30:90, 90:150], cmap="gray", vmin=0, vmax=255)
    axs[1, i].set_title(titles[i])
    axs[1, i].axis("off")
plt.show()

In [ ]:
# using OpenCV
im_1_salt_pepper_averaging_1 = cv2.filter2D(
    im_1_salt_pepper_noise, -1, kernel_average_3x3, borderType=cv2.BORDER_REFLECT
)
im_1_salt_pepper_averaging_2 = cv2.filter2D(
    im_1_salt_pepper_noise, -1, kernel_average_5x5, borderType=cv2.BORDER_REFLECT
)
im_1_salt_pepper_averaging_3 = cv2.filter2D(
    im_1_salt_pepper_noise, -1, kernel_average_9x9, borderType=cv2.BORDER_REFLECT
)

# plot
images = [
    im_1,
    im_1_salt_pepper_noise,
    im_1_salt_pepper_averaging_1,
    im_1_salt_pepper_averaging_2,
    im_1_salt_pepper_averaging_3,
]
titles = ["Original", "Salt & Pepper Noise", "3x3", "5x5", "9x9"]
fig, axs = plt.subplots(2, 5, figsize=(20, 8), layout="compressed")
for i in range(5):
    axs[0, i].imshow(images[i], cmap="gray", vmin=0, vmax=255)
    axs[0, i].set_title(titles[i])
    axs[0, i].axis("off")
    axs[1, i].imshow(images[i][30:90, 90:150], cmap="gray", vmin=0, vmax=255)
    axs[1, i].set_title(titles[i])
    axs[1, i].axis("off")
plt.show()

In [ ]:
# using Scipy
im_2_gauss_averaging_1 = np.zeros_like(im_2)
for i in range(im_2.shape[2]):
    im_2_gauss_averaging_1[:, :, i] = sp.signal.convolve2d(
        im_2_gaussian_noise[:, :, i], kernel_average_3x3, mode="same", boundary="symm"
    )

im_2_gauss_averaging_2 = np.zeros_like(im_2)
for i in range(im_2.shape[2]):
    im_2_gauss_averaging_2[:, :, i] = sp.signal.convolve2d(
        im_2_gaussian_noise[:, :, i], kernel_average_5x5, mode="same", boundary="symm"
    )

im_2_gauss_averaging_3 = np.zeros_like(im_2)
for i in range(im_2.shape[2]):
    im_2_gauss_averaging_3[:, :, i] = sp.signal.convolve2d(
        im_2_gaussian_noise[:, :, i], kernel_average_9x9, mode="same", boundary="symm"
    )

# plot
images = [im_2, im_2_gaussian_noise, im_2_gauss_averaging_1, im_2_gauss_averaging_2, im_2_gauss_averaging_3]
titles = ["Original", "Gaussian Noise", "3x3", "5x5", "9x9"]
fig, axs = plt.subplots(2, 5, figsize=(20, 8), layout="compressed")
for i in range(5):
    axs[0, i].imshow(images[i], vmin=0, vmax=255)
    axs[0, i].set_title(titles[i])
    axs[0, i].axis("off")
    axs[1, i].imshow(images[i][220:320, 220:320], cmap="gray", vmin=0, vmax=255)
    axs[1, i].set_title(titles[i])
    axs[1, i].axis("off")
plt.show()

In [ ]:
# using Scipy
kernel_average_3x3_3d = np.ones((3, 3, 1)) / 9
kernel_average_5x5_3d = np.ones((5, 5, 1)) / 25
kernel_average_9x9_3d = np.ones((9, 9, 1)) / 81

im_2_salt_pepper_averaging_1 = np.clip(
    sp.ndimage.convolve(im_2_salt_pepper_noise, kernel_average_3x3_3d, mode="constant", cval=0), 0, 255
).astype(np.uint8)
im_2_salt_pepper_averaging_2 = np.clip(
    sp.ndimage.convolve(im_2_salt_pepper_noise, kernel_average_5x5_3d, mode="constant", cval=0), 0, 255
).astype(np.uint8)
im_2_salt_pepper_averaging_3 = np.clip(
    sp.ndimage.convolve(im_2_salt_pepper_noise, kernel_average_9x9_3d, mode="constant", cval=0), 0, 255
).astype(np.uint8)

# plot
images = [
    im_2,
    im_2_salt_pepper_noise,
    im_2_salt_pepper_averaging_1,
    im_2_salt_pepper_averaging_2,
    im_2_salt_pepper_averaging_3,
]
titles = ["Original", "Gaussian Noise", "3x3", "5x5", "9x9"]
fig, axs = plt.subplots(2, 5, figsize=(20, 8), layout="compressed")
for i in range(5):
    axs[0, i].imshow(images[i], vmin=0, vmax=255)
    axs[0, i].set_title(titles[i])
    axs[0, i].axis("off")
    axs[1, i].imshow(images[i][220:320, 220:320], cmap="gray", vmin=0, vmax=255)
    axs[1, i].set_title(titles[i])
    axs[1, i].axis("off")
plt.show()

##### <a id='toc3_1_1_1_2_'></a>[Gaussian Filter](#toc0_)

- A more advanced smoothing filter that uses a Gaussian function to weight the pixels in the neighborhood.
- Weights decrease with distance from the center, giving more importance to central pixels.
- **Gaussian Function Formula**:

$$G(x, y) = \frac{1}{2\pi \sigma^2} e^{-\frac{x^2 + y^2}{2\sigma^2}}$$

- **Generalized Gaussian Mask**:

$$H_{\text{Gaussian}}(i, j) = G\left(i - \frac{N-1}{2}, j - \frac{N-1}{2}\right)$$

- **Normalization**:

$$H_{\text{Gaussian}} = \frac{1}{\sum_{i=1}^N \sum_{j=1}^N G\left(i - \frac{N-1}{2}, j - \frac{N-1}{2}\right)} \cdot G\left(i - \frac{N-1}{2}, j - \frac{N-1}{2}\right)$$

**Example:**

- $\text{kernel size} = 3$
- $\text{scale} = 0.5$

$$
H_{\text{Gaussian}} = \begin{bmatrix}
0.011 & 0.084 & 0.011 \\
0.084 & 0.619 & 0.084 \\
0.011 & 0.084 & 0.011
\end{bmatrix}_{3 \times 3}
$$


In [ ]:
# manual
def gaussian_kernel_2d(kernel_size: int, sigma: float) -> np.ndarray:
    if kernel_size % 2 == 0:
        raise ValueError("kernel size must be odd.")

    half_size = kernel_size // 2
    x = np.arange(-half_size, half_size + 1)
    y = np.arange(-half_size, half_size + 1)
    xx, yy = np.meshgrid(x, y)

    kernel = np.exp(-(xx**2 + yy**2) / (2 * sigma**2))
    kernel = kernel / np.sum(kernel)
    return kernel


# create gaussian kernels
kernel_gaussian_3x3_1 = gaussian_kernel_2d(3, sigma=0.5)
kernel_gaussian_3x3_2 = gaussian_kernel_2d(3, sigma=0.6)
kernel_gaussian_3x3_3 = gaussian_kernel_2d(3, sigma=0.8)

# apply kernels manually
im_1_gauss_gaussian_1 = cv2.filter2D(im_1_gaussian_noise, -1, kernel_gaussian_3x3_1, borderType=cv2.BORDER_CONSTANT)
im_1_gauss_gaussian_2 = cv2.filter2D(im_1_gaussian_noise, -1, kernel_gaussian_3x3_2, borderType=cv2.BORDER_CONSTANT)
im_1_gauss_gaussian_3 = cv2.filter2D(im_1_gaussian_noise, -1, kernel_gaussian_3x3_3, borderType=cv2.BORDER_CONSTANT)

# log
print(f"kernel_gaussian_3x3_1:\n{kernel_gaussian_3x3_1}\n")
print(f"kernel_gaussian_3x3_2:\n{kernel_gaussian_3x3_2}\n")
print(f"kernel_gaussian_3x3_3:\n{kernel_gaussian_3x3_3}")

# plot
images = [im_1, im_1_gaussian_noise, im_1_gauss_gaussian_1, im_1_gauss_gaussian_2, im_1_gauss_gaussian_3]
titles = ["Original", "Gaussian Noise", "gaussian 3x3", "gaussian 3x3 2", "gaussian 3x3 3"]
fig, axs = plt.subplots(2, 5, figsize=(20, 8), layout="compressed")
for i in range(5):
    axs[0, i].imshow(images[i], cmap="gray", vmin=0, vmax=255)
    axs[0, i].set_title(titles[i])
    axs[0, i].axis("off")
    axs[1, i].imshow(images[i][30:90, 90:150], cmap="gray", vmin=0, vmax=255)
    axs[1, i].set_title(titles[i])
    axs[1, i].axis("off")
plt.show()

In [ ]:
# using OpenCV
im_1_gauss_gaussian_4 = cv2.GaussianBlur(im_1_gaussian_noise, ksize=(3, 3), sigmaX=0.5)
im_1_gauss_gaussian_5 = cv2.GaussianBlur(im_1_gaussian_noise, ksize=(3, 3), sigmaX=0.6)
im_1_gauss_gaussian_6 = cv2.GaussianBlur(im_1_gaussian_noise, ksize=(3, 3), sigmaX=0.8)

# plot
images = [im_1, im_1_gaussian_noise, im_1_gauss_gaussian_4, im_1_gauss_gaussian_5, im_1_gauss_gaussian_6]
titles = ["Original", "Gaussian Noise", "gaussian 3x3", "gaussian 3x3 2", "gaussian 3x3 3"]
fig, axs = plt.subplots(2, 5, figsize=(20, 8), layout="compressed")
for i in range(5):
    axs[0, i].imshow(images[i], cmap="gray", vmin=0, vmax=255)
    axs[0, i].set_title(titles[i])
    axs[0, i].axis("off")
    axs[1, i].imshow(images[i][30:90, 90:150], cmap="gray", vmin=0, vmax=255)
    axs[1, i].set_title(titles[i])
    axs[1, i].axis("off")
plt.show()

In [ ]:
# using SciPy
# truncate = (ksize - 1) / (2 * sigma)
im_1_gauss_gaussian_7 = sp.ndimage.gaussian_filter(im_1_gaussian_noise, sigma=0.5, truncate=(3 - 1) / (2 * 0.5))
im_1_gauss_gaussian_8 = sp.ndimage.gaussian_filter(im_1_gaussian_noise, sigma=0.6, truncate=(3 - 1) / (2 * 0.6))
im_1_gauss_gaussian_9 = sp.ndimage.gaussian_filter(im_1_gaussian_noise, sigma=0.8, truncate=(3 - 1) / (2 * 0.8))

# plot
images = [im_1, im_1_gaussian_noise, im_1_gauss_gaussian_7, im_1_gauss_gaussian_8, im_1_gauss_gaussian_9]
titles = ["Original", "Gaussian Noise", "gaussian 3x3", "gaussian 3x3 2", "gaussian 3x3 3"]
fig, axs = plt.subplots(2, 5, figsize=(20, 8), layout="compressed")
for i in range(5):
    axs[0, i].imshow(images[i], cmap="gray", vmin=0, vmax=255)
    axs[0, i].set_title(titles[i])
    axs[0, i].axis("off")
    axs[1, i].imshow(images[i][30:90, 90:150], cmap="gray", vmin=0, vmax=255)
    axs[1, i].set_title(titles[i])
    axs[1, i].axis("off")
plt.show()

#### <a id='toc3_1_1_2_'></a>[Sharpening (High Pass) Filters](#toc0_)

- Used to **enhance edges** and **fine details** in the image.

✍️ **Derivative in Discrete World:**:

- **Derivatives** cannot be calculated directly in the **Discrete** world and we use **finite difference methods** to **approximate** derivatives.
- These methods include **forward difference**, **backward difference**, and **central difference**.
  - **Forward Difference**:
    - Approximates the derivative using the value at the current point and the next point.
    - First-Order Derivative:

      $$\frac{\partial f}{\partial x} \Big|_{(x,y)} = f'(x,y) \approx \frac{f(x+1, y) - f(x, y)}{1} = \begin{bmatrix}0 & 0 & 0 \\ 0 & -1 & 1 \\ 0 & 0 & 0\end{bmatrix}$$

  - **Backward Difference**:
    - Approximates the derivative using the value at the current point and the previous point.
    - First-Order Derivative:

      $$\frac{\partial f}{\partial x} \Big|_{(x-1,y)} = f'(x-1,y) \approx \frac{f(x, y) - f(x-1, y)}{1} = \begin{bmatrix}0 & 0 & 0 \\ -1 & 1 & 0 \\ 0 & 0 & 0\end{bmatrix}$$

  - **Central Difference**:
    - Approximates the derivative using the values at the previous and next points.
    - First-Order Derivative:

      $$\frac{\partial f}{\partial x} \Big|_{(x,y)} \approx \frac{f(x+1, y) - f(x-1, y)}{2 \times 1} = \begin{bmatrix}0 & 0 & 0 \\ -0.5 & 0 & 0.5 \\ 0 & 0 & 0\end{bmatrix}$$

- **Second-Order Derivative**: The second derivative is the derivative of the first derivative. Using the central difference method.

  $$\frac{\partial^2 f}{\partial x^2} \Big|_{(x,y)} \approx \frac{f'(x, y) - f'(x-1, y)}{1} = f(x+1, y) + f(x-1, y) - 2f(x, y) = \begin{bmatrix}0 & 0 & 0 \\ 1 & -2 & 1 \\ 0 & 0 & 0\end{bmatrix}$$

##### <a id='toc3_1_1_2_1_'></a>[Laplacian Filter](#toc0_)

- The Laplacian is a **second-order derivative** operator that **combines** the second derivatives in both the $x$ and $y$ directions.
- It is used for **edge detection** and **sharpening**.
- **Discrete Laplacian Formula**:

$$\nabla^2 f(x,y) = \frac{\partial^2 f}{\partial x^2} + \frac{\partial^2 f}{\partial y^2} \approx f(x+1, y) + f(x-1, y) + f(x, y+1) + f(x, y-1) - 4f(x, y)$$

**Laplacian Kernels:**

$$
\begin{array}{cccc}
\begin{bmatrix}0 & 1 & 0 \\ 1 & -4 & 1 \\ 0 & 1 & 0\end{bmatrix} &
\begin{bmatrix}1 & 1 & 1 \\ 1 & -8 & 1 \\ 1 & 1 & 1\end{bmatrix} &
\begin{bmatrix}0 & -1 & 0 \\ -1 & 4 & -1 \\ 0 & -1 & 0\end{bmatrix} &
\begin{bmatrix}-1 & -1 & -1 \\ -1 & 8 & -1 \\ -1 & -1 & -1\end{bmatrix} \\
\text{(a)} & \text{(b)} & \text{(c)} & \text{(d)}
\end{array}
$$

- **(a):** Laplacian kernel used to implement above formula.
- **(b):** Kernel used to implement an extension of this equation that includes the diagonal terms.
- **(c) & (d):** Other implementations [equivalent results, but the difference in sign must be kept in mind].


In [ ]:
# laplacian kernels
kernel_Laplacian_1 = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])
kernel_Laplacian_2 = np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]])

# apply laplacian kernels
im_1_laplacian_1 = cv2.filter2D(im_1, cv2.CV_32F, kernel_Laplacian_1, borderType=cv2.BORDER_CONSTANT)
im_1_laplacian_2 = cv2.filter2D(im_1, cv2.CV_32F, kernel_Laplacian_2, borderType=cv2.BORDER_CONSTANT)
im_1_blurred_laplacian_1 = cv2.filter2D(im_1_blurred, cv2.CV_32F, kernel_Laplacian_1, borderType=cv2.BORDER_CONSTANT)
im_1_blurred_laplacian_2 = cv2.filter2D(im_1_blurred, cv2.CV_32F, kernel_Laplacian_2, borderType=cv2.BORDER_CONSTANT)
im_1_gaussian_noise_laplacian_1 = cv2.filter2D(
    im_1_gaussian_noise, cv2.CV_32F, kernel_Laplacian_1, borderType=cv2.BORDER_CONSTANT
)
im_1_gaussian_noise_laplacian_2 = cv2.filter2D(
    im_1_gaussian_noise, cv2.CV_32F, kernel_Laplacian_2, borderType=cv2.BORDER_CONSTANT
)

# plot
images = [
    [im_1, im_1_laplacian_1, im_1_laplacian_2],
    [im_1_blurred, im_1_blurred_laplacian_1, im_1_blurred_laplacian_2],
    [im_1_gaussian_noise, im_1_gaussian_noise_laplacian_1, im_1_gaussian_noise_laplacian_2],
]
titles = [
    ["Original", "kernel_Laplacian_1", "kernel_Laplacian_2"],
    ["Blurred", "kernel_Laplacian_1", "kernel_Laplacian_2"],
    ["Gaussian Noise", "kernel_Laplacian_1", "kernel_Laplacian_2"],
]
fig, axs = plt.subplots(3, 6, figsize=(24, 8), layout="compressed")
for c in range(3):
    for r in range(3):
        axs[r, c * 2].imshow(np.abs(images[c][r]), cmap="gray", vmin=0, vmax=255)
        axs[r, c * 2].set_title(titles[c][r])
        axs[r, c * 2].axis("off")
        axs[r, c * 2 + 1].imshow(np.abs(images[c][r])[30:90, 90:150], cmap="gray", vmin=0, vmax=255)
        axs[r, c * 2 + 1].set_title(titles[c][r])
        axs[r, c * 2 + 1].axis("off")
plt.show()

In [ ]:
im_1_sharp_1 = cv2.subtract(im_1, im_1_laplacian_1, dtype=cv2.CV_8U)
im_1_sharp_2 = cv2.subtract(im_1, im_1_laplacian_2, dtype=cv2.CV_8U)
im_1_sharp_3 = cv2.subtract(im_1_blurred, im_1_blurred_laplacian_1, dtype=cv2.CV_8U)
im_1_sharp_4 = cv2.subtract(im_1_blurred, im_1_blurred_laplacian_2, dtype=cv2.CV_8U)
im_1_sharp_5 = cv2.subtract(im_1_gaussian_noise, im_1_gaussian_noise_laplacian_1, dtype=cv2.CV_8U)
im_1_sharp_6 = cv2.subtract(im_1_gaussian_noise, im_1_gaussian_noise_laplacian_2, dtype=cv2.CV_8U)

# plot
reference_images = [im_1, im_1, im_1_blurred, im_1_blurred, im_1_gaussian_noise, im_1_gaussian_noise]
reference_titles = ["Original", "Original", "Blurred", "Blurred", "Gaussian Noise", "Gaussian_Noise"]
images = [im_1_sharp_1, im_1_sharp_2, im_1_sharp_3, im_1_sharp_4, im_1_sharp_5, im_1_sharp_6]
titles = [
    "im_1_laplacian_1",
    "im_1_laplacian_2",
    "im_1_laplacian_1",
    "im_1_laplacian_2",
    "im_1_laplacian_1",
    "im_1_laplacian_2",
]
fig, axs = plt.subplots(4, 6, figsize=(24, 16), layout="compressed")
for i in range(6):
    axs[0, i].imshow(reference_images[i], cmap="gray", vmin=0, vmax=255)
    axs[0, i].set_title(reference_titles[i])
    axs[0, i].axis("off")
    axs[1, i].imshow(images[i], cmap="gray", vmin=0, vmax=255)
    axs[1, i].set_title(titles[i])
    axs[1, i].axis("off")
    axs[2, i].imshow(reference_images[i][30:90, 90:150], cmap="gray", vmin=0, vmax=255)
    axs[2, i].set_title(reference_titles[i])
    axs[2, i].axis("off")
    axs[3, i].imshow(images[i][30:90, 90:150], cmap="gray", vmin=0, vmax=255)
    axs[3, i].set_title(titles[i])
    axs[3, i].axis("off")
plt.show()

##### <a id='toc3_1_1_2_2_'></a>[Unsharp Masking](#toc0_)

- A technique that subtracts a smoothed (blurred) version of the image from the original and adds the result back to the original.
- **Formula:**

$$I_{\text{sharp}} = I_{\text{original}} + (I_{\text{original}} - I_{\text{smoothed}})$$


In [ ]:
im_1_smooth_1 = cv2.filter2D(im_1, -1, kernel_gaussian_3x3_1)
im_1_smooth_2 = cv2.filter2D(im_1, -1, kernel_gaussian_3x3_2)
im_1_smooth_3 = cv2.filter2D(im_1, -1, kernel_gaussian_3x3_3)

im_1_sub_1 = cv2.subtract(im_1, im_1_smooth_1, dtype=cv2.CV_32F)
im_1_sub_2 = cv2.subtract(im_1, im_1_smooth_2, dtype=cv2.CV_32F)
im_1_sub_3 = cv2.subtract(im_1, im_1_smooth_3, dtype=cv2.CV_32F)

im_1_sharp_7 = cv2.add(im_1, im_1_sub_1, dtype=cv2.CV_8U)
im_1_sharp_8 = cv2.add(im_1, im_1_sub_2, dtype=cv2.CV_8U)
im_1_sharp_9 = cv2.add(im_1, im_1_sub_3, dtype=cv2.CV_8U)

# plot
images = [
    [im_1, im_1_smooth_1, im_1_smooth_2, im_1_smooth_3],
    [im_1 - im_1, im_1_sub_1, im_1_sub_2, im_1_sub_3],
    [im_1, im_1_sharp_7, im_1_sharp_8, im_1_sharp_9],
]
titles = [
    ["Original", "im_1_smooth_1", "im_1_smooth_2", "im_1_smooth_3"],
    ["Original", "im_1_sub_1", "im_1_sub_2", "im_1_sub_3"],
    ["Original", "im_1_sharp_7", "im_1_sharp_8", "im_1_sharp_9"],
]
fig, axs = plt.subplots(3, 4, figsize=(16, 12), layout="compressed")
for i in range(4):
    axs[0, i].imshow(images[0][i], cmap="gray", vmin=0, vmax=255)
    axs[0, i].set_title(titles[0][i])
    axs[0, i].axis("off")
    axs[1, i].imshow(np.abs(images[1][i])[30:90, 90:150], cmap="gray", vmin=0, vmax=255)
    axs[1, i].set_title(titles[1][i])
    axs[1, i].axis("off")
    axs[2, i].imshow(images[2][i][30:90, 90:150], cmap="gray", vmin=0, vmax=255)
    axs[2, i].set_title(titles[2][i])
    axs[2, i].axis("off")
plt.show()

##### <a id='toc3_1_1_2_3_'></a>[High-Boost Filtering](#toc0_)

- A generalization of unsharp masking where the scaling factor $\mathbf{k}$ can be greater than $\mathbf{1}$.
- **Formula:**

$$I_{\text{sharp}} = I_{\text{original}} + k \cdot (I_{\text{original}} - I_{\text{smoothed}})$$


In [ ]:
im_1_smooth_1 = cv2.filter2D(im_1, -1, kernel_gaussian_3x3_1)
im_1_smooth_2 = cv2.filter2D(im_1, -1, kernel_gaussian_3x3_2)
im_1_smooth_3 = cv2.filter2D(im_1, -1, kernel_gaussian_3x3_3)

im_1_sub_1 = cv2.subtract(im_1, im_1_smooth_1, dtype=cv2.CV_32F)
im_1_sub_2 = cv2.subtract(im_1, im_1_smooth_2, dtype=cv2.CV_32F)
im_1_sub_3 = cv2.subtract(im_1, im_1_smooth_3, dtype=cv2.CV_32F)

im_1_sharp_7 = cv2.addWeighted(im_1, 1, im_1_sub_1, 2, 0, dtype=cv2.CV_8U)
im_1_sharp_8 = cv2.addWeighted(im_1, 1, im_1_sub_2, 2, 0, dtype=cv2.CV_8U)
im_1_sharp_9 = cv2.addWeighted(im_1, 1, im_1_sub_3, 2, 0, dtype=cv2.CV_8U)

# plot
images = [
    [im_1, im_1_smooth_1, im_1_smooth_2, im_1_smooth_3],
    [im_1 - im_1, np.abs(im_1_sub_1), np.abs(im_1_sub_2), np.abs(im_1_sub_3)],
    [im_1, im_1_sharp_7, im_1_sharp_8, im_1_sharp_9],
]
titles = [
    ["Original", "im_1_smooth_1", "im_1_smooth_2", "im_1_smooth_3"],
    ["Original", "im_1_sub_1", "im_1_sub_2", "im_1_sub_3"],
    ["Original", "im_1_sharp_7", "im_1_sharp_8", "im_1_sharp_9"],
]
fig, axs = plt.subplots(3, 4, figsize=(16, 12), layout="compressed")
for i in range(4):
    axs[0, i].imshow(images[0][i], cmap="gray", vmin=0, vmax=255)
    axs[0, i].set_title(titles[0][i])
    axs[0, i].axis("off")
    axs[1, i].imshow(images[1][i][30:90, 90:150], cmap="gray", vmin=0, vmax=255)
    axs[1, i].set_title(titles[1][i])
    axs[1, i].axis("off")
    axs[2, i].imshow(images[2][i][30:90, 90:150], cmap="gray", vmin=0, vmax=255)
    axs[2, i].set_title(titles[2][i])
    axs[2, i].axis("off")
plt.show()

#### <a id='toc3_1_1_3_'></a>[Edge Detection Filters](#toc0_)

- Edge detection identifies boundaries between regions of different intensities in an image.

💪 **Advantages of Linear Filters:**

- **Simplicity**: Linear filters are easy to implement and computationally efficient.
- **Effectiveness**: They work well for detecting edges in images with moderate noise.
- **Flexibility**: Different kernels (e.g., Sobel, Prewitt, Laplacian) can be used depending on the application.

⚠️ **Limitations of Linear Filters:**

- **Sensitivity to Noise**: Linear filters can amplify noise, leading to false edges.
- **Thick Edges**: First-order filters (e.g., Sobel) often produce thick edges, which may require thinning.


##### <a id='toc3_1_1_3_1_'></a>[First-Order Derivative Filters](#toc0_)

- First-order derivative filters approximate the gradient of the image, which measures the rate of change of intensity.
- The gradient is a vector:

$$\nabla f = \begin{bmatrix} G_x \\ G_y \end{bmatrix} = \begin{bmatrix} \frac{\partial f}{\partial x} \\ \frac{\partial f}{\partial y} \end{bmatrix}$$

- The magnitude of the gradient:

$$|\nabla f| = \sqrt{\left(G_x\right)^2 + \left(G_y\right)^2} \approx |G_x| + |G_y|$$

- gradient direction (orthogonal to edge direction):

$$\theta = \tan^{-1}\left(\frac{G_y}{G_x}\right)$$

🔢 **Common First-Order Filters**:

- **Roberts**
  - It uses two **2x2** kernels to approximate the gradient of an image.
  - These kernels are designed to highlight changes in intensity along the **diagonal directions** (45° and 135°).

  $$
  \begin{array}{cc}
  G_x = \begin{bmatrix} 0 & 1 \\ -1 & 0 \end{bmatrix}, \qquad
  G_y = \begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix}
  \end{array}
  $$

- **Prewitt**
  - It uses 3x3 kernels to approximate the gradient of an image.
  - It is particularly effective for detecting **horizontal** and **vertical** edges.

  $$
  \begin{array}{cc}
  G_x = \begin{bmatrix} -1 & 0 & 1 \\ -1 & 0 & 1 \\ -1 & 0 & 1 \end{bmatrix}, \qquad
  G_y = \begin{bmatrix} -1 & -1 & -1 \\ 0 & 0 & 0 \\ 1 & 1 & 1 \end{bmatrix}
  \end{array}
  $$

  - Variants of the Prewitt Filter (Diagonal):

  $$
  \begin{array}{cc}
  G_{-45^\circ} = \begin{bmatrix} -1 & -1 & 0 \\ -1 & 0 & 1 \\ 0 & 1 & 1 \end{bmatrix}, \qquad
  G_{+45^\circ} = \begin{bmatrix} 0 & 1 & 1 \\ -1 & 0 & 1 \\ -1 & -1 & 0 \end{bmatrix}
  \end{array}
  $$

- **Sobel**
  - It uses 3x3 kernels to approximate the gradient of an image.
  - It is particularly effective for detecting **horizontal** and **vertical** edges.

  $$
  \begin{array}{cc}
  G_x = \begin{bmatrix} -1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{bmatrix}, \qquad
  G_y = \begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ 1 & 2 & 1 \end{bmatrix}
  \end{array}
  $$

  - Variants of the Sobel Filter (Diagonal):

  $$
  \begin{array}{cc}
  G_{-45^\circ} = \begin{bmatrix} -2 & -1 & 0 \\ -1 & 0 & 1 \\ 0 & 1 & 2 \end{bmatrix}, \qquad
  G_{+45^\circ} = \begin{bmatrix} 0 & 1 & 2 \\ -1 & 0 & 1 \\ -2 & -1 & 0 \end{bmatrix}
  \end{array}
  $$

- **Scharr**
  - It is an improvement over the Sobel filter in terms of accuracy and rotation invariance.
  - The Scharr filter is designed to provide better results for detecting edges at small angles and is particularly useful in applications requiring high precision.

  $$
  \begin{array}{cc}
  G_x = \begin{bmatrix} -3 & 0 & 3 \\ -10 & 0 & 10 \\ -3 & 0 & 3 \end{bmatrix}, \qquad
  G_y = \begin{bmatrix} -3 & -10 & -3 \\ 0 & 0 & 0 \\ 3 & 10 & 3 \end{bmatrix}
  \end{array}
  $$

🪜 **Edge Detection Steps:**

1. Compute Gradient Components
1. Compute Gradient Magnitude
1. Compute Edge Direction (Optional)
1. Threshold the Gradient Magnitude


📝 **Docs**:

- Roberts Cross Filters **[PhD thesis]** in *1963*: [Machine Perception of Three-Dimensional Solids](https://dspace.mit.edu/bitstream/handle/1721.1/11589/33959125-MIT.pdf)
- Prewitt Filters **[Book]** in *1970* :[Object Enhancement and Extraction](https://web.eecs.utk.edu/~hqi/ece472-572/reference/edge-Prewitt70.pdf)
- Sobel Filters **[Paper]** in *1968* :[A 3×3 Isotropic Gradient Operator for Image Processing](https://www.researchgate.net/publication/285159837_A_33_isotropic_gradient_operator_for_image_processing)
- Scharr FIlters **[PhD thesis]** in *2000*: [Optimale Operatoren in der digitalen Bildverarbeitung](https://www.researchgate.net/publication/33427401_Optimale_Operatoren_in_der_Digitalen_Bildverarbeitung)


In [ ]:
# forward difference (ignoring last row and column)
g_x_forward = np.zeros(im_1.shape, dtype=np.float32)
g_y_forward = np.zeros(im_1.shape, dtype=np.float32)
g_x_forward[:, :-1] = np.abs(cv2.subtract(im_1[:, 1:], im_1[:, :-1], dtype=cv2.CV_32F))
g_y_forward[:-1, :] = np.abs(cv2.subtract(im_1[1:, :], im_1[:-1, :], dtype=cv2.CV_32F))

# compute gradient magnitude
grad_mag_forward = np.sqrt(g_x_forward**2 + g_y_forward**2)

# normalize and convert to uint8
grad_mag_forward = np.clip(grad_mag_forward, 0, 255).astype(np.uint8)

# plot
images = [im_1, g_x_forward, g_y_forward, grad_mag_forward]
titles = ["Original", "Gx", "Gy", "Magnitude"]
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
for i in range(4):
    axs[i].imshow(images[i], cmap="gray", vmin=0, vmax=255)
    axs[i].set_title(titles[i])
    axs[i].axis("off")
plt.show()

In [ ]:
# central difference (ignoring first and last row/column)
g_x_center = np.zeros(im_1.shape, dtype=np.float32)
g_y_center = np.zeros(im_1.shape, dtype=np.float32)
g_x_center[:, 1:-1] = np.abs(cv2.subtract(im_1[:, 2:], im_1[:, :-2], dtype=cv2.CV_32F) / 2.0)
g_y_center[1:-1, :] = np.abs(cv2.subtract(im_1[2:, :], im_1[:-2, :], dtype=cv2.CV_32F) / 2.0)

# compute gradient magnitude
grad_mag_center = np.sqrt(g_x_center**2 + g_y_center**2)

# normalize and convert to uint8
grad_mag_center = np.clip(grad_mag_center, 0, 255).astype(np.uint8)

# plot
images = [im_1, g_x_center, g_y_center, grad_mag_center]
titles = ["Original", "Gx", "Gy", "Magnitude"]
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
for i in range(4):
    axs[i].imshow(images[i], cmap="gray", vmin=0, vmax=255)
    axs[i].set_title(titles[i])
    axs[i].axis("off")
plt.show()

In [ ]:
# roberts cross kernels
roberts_x = np.array([[0, 1], [-1, 0]])
roberts_y = np.array([[1, 0], [0, -1]])

# convolve with the Roberts kernels
g_x_roberts = cv2.filter2D(im_1, cv2.CV_32F, roberts_x)
g_y_roberts = cv2.filter2D(im_1, cv2.CV_32F, roberts_y)

# compute gradient magnitude and angle
grad_roberts_mag = np.sqrt(g_x_roberts**2 + g_y_roberts**2)
grad_roberts_angle = np.arctan2(g_y_roberts, g_x_roberts)  # in radians

# normalize and convert to uint8
grad_roberts_mag = np.clip(grad_roberts_mag, 0, 255).astype(np.uint8)
grad_roberts_angle = (grad_roberts_angle + np.pi) / (2 * np.pi) * 255

# apply a threshold using Otsu's method
# _, threshold_value = cv2.threshold(grad_roberts_mag, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
# grad_roberts_mag = np.where(grad_roberts_mag >= threshold_value, grad_roberts_mag, 0)

# plot
images = [im_1, np.abs(g_x_roberts), np.abs(g_y_roberts), grad_roberts_mag, grad_roberts_angle]
titles = ["Original", "Gx", "Gy", "Magnitude", "Direction (Angle)"]
fig, axs = plt.subplots(1, 5, figsize=(20, 4), layout="compressed")
for i in range(5):
    axs[i].imshow(images[i], cmap="gray", vmin=0, vmax=255)
    axs[i].set_title(titles[i])
    axs[i].axis("off")
plt.show()

In [ ]:
# prewitt kernels
prewitt_x = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
prewitt_y = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])

# convolve with the prewitt kernels
g_x_prewitt = cv2.filter2D(im_1, cv2.CV_32F, prewitt_x)
g_y_prewitt = cv2.filter2D(im_1, cv2.CV_32F, prewitt_y)

# compute gradient magnitude and angle
grad_prewitt_mag = np.sqrt(g_x_prewitt**2 + g_y_prewitt**2)
grad_prewitt_angle = np.arctan2(g_y_prewitt, g_x_prewitt)  # in radians

# normalize and convert to uint8
grad_prewitt_mag = np.clip(grad_prewitt_mag, 0, 255).astype(np.uint8)
grad_prewitt_angle = (grad_prewitt_angle + np.pi) / (2 * np.pi) * 255

# apply a threshold using Otsu's method
_, threshold_value = cv2.threshold(grad_prewitt_mag, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
grad_prewitt_mag = np.where(grad_prewitt_mag >= threshold_value, grad_prewitt_mag, 0)

# plot
images = [im_1, np.abs(g_x_prewitt), np.abs(g_y_prewitt), grad_prewitt_mag, grad_prewitt_angle]
titles = ["Original", "Gx", "Gy", "Magnitude", "Direction (Angle)"]
fig, axs = plt.subplots(1, 5, figsize=(20, 4), layout="compressed")
for i in range(5):
    axs[i].imshow(images[i], cmap="gray", vmin=0, vmax=255)
    axs[i].set_title(titles[i])
    axs[i].axis("off")
plt.show()

In [ ]:
# sobel kernels
sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
sobel_d1 = np.array([[-2, -1, 0], [-1, 0, 1], [0, 2, 1]])
sobel_d2 = np.array([[0, 1, 2], [-1, 0, 1], [-2, -1, 0]])

# convolve with the sobel kernels
g_x_sobel = np.abs(cv2.filter2D(im_1, cv2.CV_32F, sobel_x))
g_y_sobel = np.abs(cv2.filter2D(im_1, cv2.CV_32F, sobel_y))
g_d1_sobel = np.abs(cv2.filter2D(im_1, cv2.CV_32F, sobel_d1))
g_d2_sobel = np.abs(cv2.filter2D(im_1, cv2.CV_32F, sobel_d2))

# compute gradient magnitude
grad_sobel_mag = np.sqrt(g_x_sobel**2 + g_y_sobel**2 + g_d1_sobel**2 + g_d2_sobel**2)

# normalize and convert to uint8
grad_sobel_mag = np.clip(grad_sobel_mag, 0, 255).astype(np.uint8)

# apply a threshold using Otsu's method
_, threshold_value = cv2.threshold(grad_sobel_mag, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
grad_sobel_mag = np.where(grad_sobel_mag >= threshold_value, grad_sobel_mag, 0)

# plot
images = [im_1, g_x_sobel, g_y_sobel, g_d1_sobel, g_d2_sobel, grad_sobel_mag]
titles = ["Original", "Gx", "Gy", "Gd1", "Gd2", "Magnitude"]
fig, axs = plt.subplots(1, 6, figsize=(24, 4), layout="compressed")
for i in range(6):
    axs[i].imshow(images[i], cmap="gray", vmin=0, vmax=255)
    axs[i].set_title(titles[i])
    axs[i].axis("off")
plt.show()

##### <a id='toc3_1_1_3_2_'></a>[Second-Order Derivative Filters](#toc0_)

-  Unlike First-Order Derivative Filters (like Sobel, Prewitt, and Roberts), which measure gradient strength and direction.
-  Second-Order Derivative filters enhance edge detection by highlighting areas where intensity changes most rapidly.

🔢 **Common First-Order Filters**:

- **Laplacian Operator**

$$\begin{bmatrix}0 & 1 & 0 \\ 1 & -4 & 1 \\ 0 & 1 & 0\end{bmatrix}$$

  - **Variants of the Laplacian Filter**
    - Laplacian with Diagonals
      - fdf

      $$\begin{bmatrix}1 & 1 & 1 \\ 1 & -8 & 1 \\ 1 & 1 & 1\end{bmatrix}$$

    - Laplacian of Gaussian (LoG)
      - Since the Laplacian is highly sensitive to noise, we often smooth the image first using a Gaussian filter.
      
      $$LoG(x,y) = -\frac{1}{\pi \sigma^4} \left(1 - \frac{x^2 + y^2}{2\sigma^2}\right) e^{-\frac{x^2 + y^2}{2\sigma^2}} = \nabla^2 (G(x,y) * I(x,y))$$

    - Difference  of Gaussian (DoG)
      - The Difference of Gaussians (DoG) is an approximation of the LoG and is computed as:

      $$DoG = G_{\sigma_1} - G_{\sigma_2}$$


### <a id='toc3_1_2_'></a>[Nonlinear Filters](#toc0_)


#### <a id='toc3_1_2_1_'></a>[Order-Statistic Filters](#toc0_)

- These filters are based on the **ranking** (ordering) of pixel values in a neighborhood.
- They are widely used for tasks like **noise reduction** and **edge detection**.


##### <a id='toc3_1_2_1_1_'></a>[Median Filter](#toc0_)

- Replaces each pixel with the **median** value of its neighborhood.
- Highly effective for removing **salt-and-pepper** noise.
- Preserves edges better than linear smoothing filters.


In [ ]:
# manual
def median_filter_2d(image: np.ndarray, kernel_size: int) -> np.ndarray:
    image_height, image_width = image.shape
    pad_size = kernel_size // 2
    padded_image = np.pad(image, pad_size, mode="constant", constant_values=0)
    output_image = np.zeros_like(image, dtype=np.uint8)

    for i in range(image_height):
        for j in range(image_width):
            roi = padded_image[i : i + kernel_size, j : j + kernel_size]
            output_image[i, j] = np.median(roi)

    return output_image


im_1_salt_pepper_median_1 = median_filter_2d(im_1_salt_pepper_noise, 3)

# plot
images = [im_1, im_1_salt_pepper_noise, im_1_salt_pepper_averaging_1, im_1_salt_pepper_median_1]
titles = ["Original", "Salt & Pepper Noise", "Averaging 3x3", "Median 3x3"]
fig, axs = plt.subplots(2, 4, figsize=(16, 8), layout="compressed")
for i in range(4):
    axs[0, i].imshow(images[i], cmap="gray", vmin=0, vmax=255)
    axs[0, i].set_title(titles[i])
    axs[0, i].axis("off")
    axs[1, i].imshow(images[i][30:90, 90:150], cmap="gray", vmin=0, vmax=255)
    axs[1, i].set_title(titles[i])
    axs[1, i].axis("off")
plt.show()

In [ ]:
# using OpenCV
im_1_salt_pepper_median_2 = cv2.medianBlur(im_1, ksize=3)

# plot
images = [im_1, im_1_salt_pepper_noise, im_1_salt_pepper_averaging_1, im_1_salt_pepper_median_2]
titles = ["Original", "Salt & Pepper Noise", "Averaging 3x3", "Median 3x3"]
fig, axs = plt.subplots(2, 4, figsize=(16, 8), layout="compressed")
for i in range(4):
    axs[0, i].imshow(images[i], cmap="gray", vmin=0, vmax=255)
    axs[0, i].set_title(titles[i])
    axs[0, i].axis("off")
    axs[1, i].imshow(images[i][30:90, 90:150], cmap="gray", vmin=0, vmax=255)
    axs[1, i].set_title(titles[i])
    axs[1, i].axis("off")
plt.show()

##### <a id='toc3_1_2_1_2_'></a>[Maximum and Minimum Filters](#toc0_)

- Replaces each pixel with the **minimum** or **maximum** value of its neighborhood.
- **Minimum** is useful for removing **white noise** (e.g., salt noise).
- **Maximum** is useful for removing **black noise** (e.g., pepper noise).

##### <a id='toc3_1_2_1_3_'></a>[Percentile Filters](#toc0_)

- Replaces each pixel with a specific **percentile** value (e.g., 75th percentile) from the sorted neighborhood.


#### <a id='toc3_1_2_2_'></a>[Bilateral Filter](#toc0_)

- The bilateral filter is a nonlinear filter that **smooths** an image while **preserving edges**.
- It combines **spatial proximity** and **intensity similarity** to determine the weights for filtering.

**Mathematical Formulation:**
- Filtered Value:

  $$I_{\text{filtered}}(x, y) = \frac{1}{W} \sum_{i,j \in \Omega} w_s(i, j) \cdot w_r(i, j) \cdot I(i, j)$$

- Normalization Factor:

  $$W = \sum_{i,j \in \Omega} w_s(i, j) \cdot w_r(i, j)$$

- Range Weight:
  - It is a Gaussian function of the intensity difference between $I(x, y)$ and $I(i, j)$.

  $$w_r(i, j) = e^{-\frac{(I(x, y) - I(i, j))^2}{2\sigma_r^2}}$$

- Spatial Weight:
  - It is a Gaussian function of the Euclidean distance between $(x, y)$ and $(i, j)$.

  $$w_s(i, j) = e^{-\frac{(x-i)^2 + (y-j)^2}{2\sigma_s^2}}$$

**Example:**
- Range Weight Scale : $\sigma_r = 50$
- Spatial Weight Scale : $\sigma_s = 1$

$$
\begin{array}{cccc}
I = \begin{bmatrix}10 & 20 & 10 \\ 20 & 255 & 20 \\ 10 & 20 & 10\end{bmatrix}, &
w_r \approx \begin{bmatrix} e^{-12.25} & e^{-5.78} & e^{-12.25} \\ e^{-5.78} & 1 & e^{-5.78} \\ e^{-12.25} & e^{-5.78} & e^{-12.25} \end{bmatrix}, &
w_s = \begin{bmatrix} e^{-1} & e^{-0.5} & e^{-1} \\ e^{-0.5} & 1 & e^{-0.5} \\ e^{-1} & e^{-0.5} & e^{-1} \end{bmatrix},
W \approx 1.0076 &
\end{array}
$$


#### <a id='toc3_1_2_3_'></a>[Canny Edge Detector](#toc0_)

- It is a multi-step algorithm used to detect edges in images.
- It is one of the most popular edge detection methods because it is:
  - **Accurate**: Detects true edges while suppressing noise.
  - **Robust**: Uses hysteresis thresholding to reduce false edges.
  - **Efficient**: Combines multiple steps into a single pipeline.

🪜 **Steps of the Canny Edge Detector**:
- **Noise Reduction**:
  - Apply a Gaussian blur to smooth the image and reduce noise.
  - This step ensures that the algorithm is less sensitive to small variations in intensity.
- **Gradient Calculation**:
  - Compute the intensity gradients (using Sobel filters) to find the magnitude and direction of edges.
- **Non-Maximum Suppression**:
  - Thin out the edges by keeping only the local maxima in the gradient magnitude image (for each pixel, check the two neighboring pixels along the gradient direction).
- **Double Thresholding**:
  - Use two thresholds (low and high) to classify edges as strong, weak, or non-edges.
- **Edge Tracking by Hysteresis**:
  - Finalize edges by connecting weak edges to strong edges, ensuring continuity.

📝 **Docs**:

- `cv2.canny`: [docs.opencv.org/master/dd/d1a/group__imgproc__feature.html#ga2a671611e104c093843d7b7fc46d24af](https://docs.opencv.org/master/dd/d1a/group__imgproc__feature.html#ga2a671611e104c093843d7b7fc46d24af)


In [ ]:
canny_edges = cv2.Canny(cv2.GaussianBlur(im_1, (3, 3), 0), 50, 150)

In [ ]:
# plot
images = [im_1, grad_roberts_mag, grad_prewitt_mag, grad_sobel_mag, canny_edges]
titles = ["Original", "Roberts", "Prewitt", "Sobel", "Canny"]
fig, axs = plt.subplots(2, 5, figsize=(20, 8), layout="compressed")
for i in range(5):
    axs[0, i].imshow(images[i], cmap="gray", vmin=0, vmax=255)
    axs[0, i].set_title(titles[i])
    axs[0, i].axis("off")
    axs[1, i].imshow(images[i][30:90, 90:150], cmap="gray", vmin=0, vmax=255)
    axs[1, i].set_title(titles[i])
    axs[1, i].axis("off")
plt.show()

#### <a id='toc3_1_2_4_'></a>[Adaptive Filtering](#toc0_)

- Adaptive filters adjust their behavior based on **local image characteristics**, such as noise level or edge strength.
- They are particularly useful for handling **non-uniform** noise.

**Types of Adaptive Filters:**
- Adaptive Median Filter
- Adaptive Wiener Filter
- Adaptive Bilateral Filter
